# create_elementMap( )

Creates a dictionary of the form 
   'X': '<Element Name>''
    where,
    X is the symbol of the element,
    <Element Name> is the common name of the element
    
    For example, 
    'H': 'hydrogen'
    
    Returns:
    created dictionary object

In [2]:
def create_elementMap():
    '''
    Creates a dictionary of the form 
    'X': '<Element Name>''
    where,
    X is the symbol of the element,
    <Element Name> is the common name of the element
    
    For example, 
    'H': 'hydrogen'
    
    Returns:
    created dictionary object
    
    '''
    
    symbol_list = []

    for index in range(1,119):
        obj = element(index)
        symbol_list.append(obj.symbol)

    #mod_symbol_list = ['Element ' + sym for sym in symbol_list]

    #data = pd.read_csv('../DataExtraction/LookupTable.csv', sep='\t')
    #data.set_index('Properties', inplace=True)
    
    element_List = list(element_data.columns)
    
    elementMap = dict(zip(symbol_list, element_List))
    
    return elementMap

In [ ]:
#Creating a dictionary to map elements to the property table
element_data = pd.read_csv('./small_LookupTable.csv', sep='\t')
element_data.set_index('Properties', inplace=True)

elementMap = create_elementMap()

# Arithmetic Function block

In [3]:
def wt_gmean(values, weights):
    wt_gm = 1
    for index, item in enumerate(values):
        wt_gm = wt_gm * pow(item, weights[index])
    
    return wt_gm

def wt_std(values, weights):
    wt_stdev = 0
    mu = np.average(values)
    for index, item in enumerate(values):
        temp = weights[index]*pow((values[index] - mu),2)
        wt_stdev = wt_stdev + temp

    wt_stdev = sqrt(wt_stdev)
    
    return wt_stdev

# create_features(chem_compostion)
    creates the feature vector for each chemical composition provided i. e.
    list of characteristics such as number of elements and 
    the statistics of the following properties:
    
    atomic_mass, fie, atomic_radius, Density, ElectronAffinity, FusionHeat, 
    ThermalConductivity, mean_Valence
    
    following statistics are calculated for each of the properties except number_of_elements:
    
    wtd_mean, gmean, wtd_gmean, entropy, wtd_entropy, range, wtd_range,
    std, wtd_std
    
    returns: a list of 81 float objects for each compound

In [ ]:
def create_features(chem_compostion):
    '''
    creates the feature vector for each chemical composition provided i. e.
    list of characteristics such as number of elements and 
    the statistics of the following properties:
    
    atomic_mass, fie, atomic_radius, Density, ElectronAffinity, FusionHeat, 
    ThermalConductivity, mean_Valence
    
    following statistics are calculated for each of the properties except number_of_elements:
    
    wtd_mean, gmean, wtd_gmean, entropy, wtd_entropy, range, wtd_range,
    std, wtd_std
    
    returns: a list of float objects
    '''
    comp = dict(chem_compostion)
    
    # creating readable compound dictionary
    ele_list = list(dict(comp).keys())
    ele_list = [str(item) for item in ele_list]

    comp_values = list(dict(comp).values())
    
    element_names = [elementMap[ky] for ky in ele_list]
    
    compound = dict(zip(element_names, comp_values))
    
    # Property List
    prop_list = ['AtomicMass', 'IonizationEnergy', 'AtomicRadius', 'Density', 'ElectronAffinity', 'FusionHeat', 'ThermalConductivity', 'Valence']
    
     # Extracting Properties
    prop_array = np.zeros((len(prop_list), len(ele_list)))

    index = 0
    for item2 in prop_list:
        prop_values = []
        for item1 in element_names:
            if item2 == 'IonizationEnergy':
                #print(item2, item1)
                #print(element_data.at[item2, item1])
                prop_values.append(literal_eval(element_data.at[item2, item1])[0])
            else:
                #print(item2, item1)
                #print(element_data.at[item2, item1])
                prop_values.append(literal_eval(element_data.at[item2, item1]))

        prop_array[index] = np.array(prop_values)
        index = index + 1

    
    # some important measures
    p_ratio = np.fromiter((item/sum(comp_values) for item in comp_values), float)
    
    w_fraction = np.zeros((len(prop_list), len(ele_list)))
    
    for index1 in range(w_fraction.shape[0]):
        for index2 in range(w_fraction.shape[1]):
            w_fraction[index1][index2] = prop_array[index1][index2]/np.sum(prop_array[index1])
    
    inter_value_matrix = np.zeros((len(prop_list), len(ele_list)))

    for index1 in range(w_fraction.shape[0]):
        inter_value_matrix[index1] = p_ratio*w_fraction[index1]/np.sum(p_ratio*w_fraction[index1])

    # Final Feature Vector
    f_Vec = []

    #no_of_elements
    f_Vec.append(len(element_names))

    for index1 in range(prop_array.shape[0]):
        # mean
        f_Vec.append(np.average(prop_array[index1]))
        #wt_mean
        f_Vec.append(np.average(prop_array[index1], weights=p_ratio))
        #gmean
        f_Vec.append(gmean(prop_array[index1]))
        #wt_gmean
        f_Vec.append(wt_gmean(prop_array[index1], p_ratio))
        #entropy
        f_Vec.append(entropy(w_fraction[index1]))
        #wt_entropy
        f_Vec.append(entropy(inter_value_matrix[index1]))

        val_max = np.max(prop_array[index1])
        val_min = np.min(prop_array[index1])

        # arg_wt_min = np.argmin(prop_array[index1])
        # arg_wt_max = np.argmax(prop_array[index1])

        wt_range_cal = p_ratio*prop_array[index1]
    
        wt_val_max = np.max(wt_range_cal)
        wt_val_min = np.min(wt_range_cal)

        #range
        f_Vec.append(val_max - val_min)
        #wt_range
        f_Vec.append(wt_val_max - wt_val_min)
        #std
        f_Vec.append(np.std(prop_array[index1]))
        #wt_std
        f_Vec.append(wt_std(prop_array[index1], p_ratio))
        
    return f_Vec


# set_plot_params()
Function to set font sizes for key objects in a matplotlib plot object

In [1]:
def set_plot_param(plt, small_size, medium_size, large_size):
    
    plt.rc('font', size=small_size)          # controls default text sizes
    plt.rc('axes', titlesize=medium_size)     # fontsize of the axes title
    plt.rc('axes', labelsize=medium_size)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=small_size)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=small_size)    # fontsize of the tick labels
    plt.rc('legend', fontsize=small_size)    # legend fontsize
    plt.rc('figure', titlesize=large_size)  # fontsize of the figure title
    
    return plt